This notebook loads behavioral data from the Hidden Multiplier Trust Game fMRI experiment and fits the different utility models described in the paper to each participant's dataset. The model fitting jobs are sent in parallel to the computing cluster, which runs the Python scripts and functions located in the Functions folder. Refer to those scripts to see the actual model code. Results are stored in Results/1.Behavior-FitModels.

In [1]:
%matplotlib inline
import os, glob, warnings, sys
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.optimize import least_squares
from scipy.io import loadmat

sys.path.append('/project/3014018.02/analysis_mri/'+
                'DataSharingCollection/Code/1.Behavior-FitModels/Functions/')
import choiceModels, costFunctions, penalizedModelFit

base_dir = '/project/3014018.02/analysis_mri/DataSharingCollection'

In [ ]:
import subprocess
subNums = np.array(pd.read_csv(os.path.join(base_dir,'Data/subjectsIncluded_batch1.csv'),header=None).T.append(
            pd.read_csv(os.path.join(base_dir,'Data/subjectsIncluded_batch2.csv'),header=None).T)).T[0]
print subNums
print len(np.unique(subNums))
niter = 100
jobs_iterations = np.arange(1,11,1)
for jobs_iteration in jobs_iterations:
    if os.path.isdir(os.path.join(base_dir,'Results/1.Behavior-FitModels/Iteration_%i'%jobs_iteration))==False:
        os.mkdir(os.path.join(base_dir,'Results/1.Behavior-FitModels/Iteration_%i'%jobs_iteration))
    jobids = pd.DataFrame(columns=['sub','jobid'])
    for sub in subNums:
        if ~os.path.isfile(os.path.join(base_dir,
                    'Results/1.FitModel/Iteration_%i/Results_sub-%03d.csv'%(jobs_iteration,sub))):
            cmd = ['/home/decision/jervbaa/.conda/envs/hmtg_fmri_nc/bin/python2.7',
                '/home/decision/jervbaa/Software/SubmitToCluster.py',
                '-length','2:30:00',
                '-memory','2GB',
                '-name','Fit_MP_model-%02d-%i'%(sub,jobs_iteration),
                '-logfiledir',os.path.join(base_dir,'Results/1.Behavior-FitModels/Logfiles/'),
                '-command','python2.7 %s/Code/1.Behavior-FitModels/Functions/fitModels.py %i %i %i'%(
                    base_dir,sub,niter,jobs_iteration),
                ]
            out = subprocess.check_output(' '.join(cmd),shell=True)
            print out
            jobid = out[-27:-1]
            tmp = pd.DataFrame(columns=jobids.columns)
            tmp.loc[0,'sub'] = sub
            tmp.loc[0,'jobid'] = jobid
            jobids = jobids.append(tmp)
    jobids.to_csv(os.path.join(base_dir,'Results/1.Behavior-FitModels/Logfiles/jobids_iteration_%i.csv'%jobs_iteration))

## Aggregate job iterations

In [4]:
subNums = np.array(pd.read_csv(os.path.join(base_dir,'Data/subjectsIncluded_batch1.csv'),header=None).T.append(
            pd.read_csv(os.path.join(base_dir,'Data/subjectsIncluded_batch2.csv'),header=None).T)).T[0]
print subNums
print len(np.unique(subNums))
mean_bests_all = pd.DataFrame()
first_bests_all = pd.DataFrame()
for sub in subNums:
    results = pd.DataFrame()
    for iteration in np.arange(1,11,1):
        results = results.append(pd.read_csv(os.path.join(base_dir,
                     'Results/1.Behavior-FitModels/Iteration_%i/Results_sub-%03d.csv'%(iteration,sub)),index_col=0))
    for model in ['GR','GA','GA_ppSOE','IA','MP','MP_ppSOE']:
        bestSSE = np.min(results.loc[results['model']==model,'SSE'])
        bests = results.loc[(results['SSE'] == bestSSE) & (results['model']==model),:]
        mean_bests_all = mean_bests_all.append(bests.groupby(['sub','model']).mean().reset_index())
        first_bests_all = first_bests_all.append([bests.iloc[-1,:]])
mean_bests_all = mean_bests_all.reset_index(drop=True)
first_bests_all = first_bests_all.reset_index(drop=True)
mean_bests_all.to_csv(os.path.join(base_dir,'Results/1.Behavior-FitModels/Mean_bests_all.csv'))
first_bests_all.to_csv(os.path.join(base_dir,'Results/1.Behavior-FitModels/First_bests_all.csv'))

[124 126 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 148 150 151 152 153 154 156 157 158 159 160 161 162 164 165
 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 189
 190 191 192]
57
